In [1]:
import os
import pandas as pd
import plotly.express as px
import dask.dataframe as ddf

from notnews import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Bashar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Bashar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
FILE_PATH = '../dataverse_files/'
PROCESSED_PATH = FILE_PATH + 'processed/'

In [3]:
def split_text(s, n):
    pieces = str(s).split()
    return (' '.join(pieces[i:i+n]) for i in range(0, len(pieces), n))

In [4]:
def split_df(df, column, num=500):
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        w_generator = split_text(presplit, num)
        for word in w_generator:
            indexes.append(i)
            new_values.append(word)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df

In [ ]:
big_dd = pd.DataFrame()
for file in os.listdir('../dataverse_files/'):
    if file.endswith(".csv"):
        print (file)
        dd = pd.read_csv( FILE_PATH + str(file), encoding = 'ISO-8859-1')
        print (f'Processing: {FILE_PATH + str(file)} ....')
        split_dd = split_df(dd, 'text')
        print (f'Finished splitting {FILE_PATH + str(file)} into chunks')
        soft_news_df = pred_soft_news_us(split_dd, col='text')
        print (f'Saving results for {FILE_PATH + str(file)}\n\n')
        soft_news_df.to_csv('{}soft_news_{}'.format(PROCESSED_PATH, file))
        os.rename(FILE_PATH+file, PROCESSED_PATH+'soft_news_'+file)

cnn-1.csv
Processing: ../dataverse_files/cnn-1.csv ....
Finished splitting ../dataverse_files/cnn-1.csv into chunks
Using model data from /Users/Bashar/opt/anaconda3/envs/nonconform/lib/python3.8/site-packages/notnews/data/us_model/nyt_us_soft_news_classifier.joblib...
Using vectorizer data from /Users/Bashar/opt/anaconda3/envs/nonconform/lib/python3.8/site-packages/notnews/data/us_model/nyt_us_soft_news_vectorizer.joblib...
Loading the model and vectorizer data file...
Saving results for ../dataverse_files/cnn-1.csv


cnn-2.csv
Processing: ../dataverse_files/cnn-2.csv ....
Finished splitting ../dataverse_files/cnn-2.csv into chunks
Saving results for ../dataverse_files/cnn-2.csv


cnn-3.csv
Processing: ../dataverse_files/cnn-3.csv ....
Finished splitting ../dataverse_files/cnn-3.csv into chunks
